In [2]:
import pandas as pd 
df = pd.read_csv('data.csv');

In [3]:
df

,deliveryId,timestamp,valid,clientIp,adRelType,noAdReasonCode,forcedAd,testAd,uid,dnt,...,status,day_of_week,h,m,s,firstEventTime,lastEventTime,eventCount,distEventTypes,eventTypeCounts
0,106434215248,2016-04-05 04:18:20.513,False,103.245.204.35,2,0.0,False,False,929546703794044818,0,...,Fraud,1,4,18,20,1970-01-01 00:00:00.000,1970-01-01 00:00:00.000,0,0,0
1,106434215253,2016-04-05 04:18:20.527,False,103.245.204.35,2,0.0,False,False,929546703794044818,0,...,Fraud,1,4,18,20,1970-01-01 00:00:00.000,1970-01-01 00:00:00.000,0,0,0
2,106439032776,2016-04-05 05:25:44.194,False,103.245.204.35,2,0.0,False,False,950040168470207184,0,...,Fraud,1,5,25,44,1970-01-01 00:00:00.000,1970-01-01 00:00:00.000,0,0,0
3,106439032787,2016-04-05 05:25:44.212,False,103.245.204.35,2,0.0,False,False,950040168470207184,0,...,Fraud,1,5,25,44,1970-01-01 00:00:00.000,1970-01-01 00:00:00.000,0,0,0
4,106439032796,2016-04-05 05:25:44.227,False,103.245.204.35,2,0.0,False,False,950040168470207184,0,...,Fraud,1,5,25,44,1970-01-01 00:00:00.000,1970-01-01 00:00:00.000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276560,109553065240,2016-04-25 07:12:51.759,True,46.208.37.225,1,0.0,False,False,1438617964128982148,0,...,Fraud,0,7,12,51,2016-04-25 07:12:52.022,2016-04-25 07:12:52.063,2,2,"{4004: 1, 4008: 1}"
276561,109553065240,2016-04-25 07:12:51.759,True,46.208.37.225,1,0.0,False,False,1438617964128982148,0,...,Fraud,0,7,12,51,2016-04-25 07:12:52.291,2016-04-25 07:12:52.429,2,2,"{4005: 1, 4006: 1}"
276562,109553190403,2016-04-25 07:13:59.242,True,62.232.113.234,1,0.0,False,False,5424024620777563043,0,...,Fraud,0,7,13,59,2016-04-25 07:14:04.785,2016-04-25 07:14:04.785,1,1,{30: 1}
276563,109553361839,2016-04-25 07:15:29.869,True,109.153.131.13,1,0.0,False,False,3518370183087839652,0,...,Fraud,0,7,15,29,1970-01-01 00:00:00.000,1970-01-01 00:00:00.000,0,0,0


In [4]:
import numpy as np 
np.random.seed(5) 

In [5]:
from sklearn import decomposition; 
from sklearn import datasets; 
pca = decomposition.PCA(n_components=52); 

In [6]:
# only the attributes not class 
df_lbls = df.copy(); 
del df['status']

In [7]:
# convert time to sin/cos lines 
def convert_time(col):
    import time 
    import datetime 
    seconds_in_day = 24*60*60
    timestamps = [] 
    for i in col:
        timestamps.append(time.mktime(datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S.%f").timetuple()))
    timestamps = np.array(timestamps)
    sinT = np.sin(2*np.pi*timestamps/seconds_in_day)
    cosT = np.cos(2*np.pi*timestamps/seconds_in_day)
    return sinT, cosT 


# drop timestamp cols and then overrite with sin/cos times 
sin_timestamp, cos_timestamp = convert_time(df.timestamp); 
sin_fet, cos_fet = convert_time(df.firstEventTime); 
sin_let, cos_let = convert_time(df.lastEventTime); 
sin_uid, cos_uid = convert_time(df.uidCreationTimestamp); 

del df['timestamp'] 
del df['firstEventTime'] 
del df['lastEventTime'] 
del df['uidCreationTimestamp'] 

df['sin_time'] = sin_timestamp; 
df['cos_time'] = cos_timestamp; 
df['sin_fet'] = sin_fet;
df['cos_fet'] = cos_fet; 
df['sin_let'] = sin_let; 
df['cos_let'] = cos_let; 
df['sin_uid'] = sin_uid; 
df['cos_uid'] = cos_uid; 
c = ((52-4)+8)
pca = decomposition.PCA(n_components=c) 
 


In [8]:
# encoding the ip addresses to digits 
import socket
import struct
def ip2int(col):
    addresses = [] 
    for addr in col:
        addresses.append(struct.unpack("!I", socket.inet_aton(addr))[0])
    return addresses; 

clientIps = ip2int(df.clientIp)
del df['clientIp']; 
df['clientIp'] = clientIps

In [9]:
del df['browserVersion']
df.columns


Index(['deliveryId', 'valid', 'adRelType', 'noAdReasonCode', 'forcedAd',
       'testAd', 'uid', 'dnt', 'marketId', 'operatorId', 'surfSessionId',
       'adSpaceId', 'accountid', 'siteid', 'clientId', 'clientVersionId',
       'webViewSdkVersion', 'unknownDeviceId', 'osId', 'platformId',
       'deviceModelId', 'platformTypeId', 'browserId', 'ipMarketId',
       'ipCountryCode', 'ipRegion', 'ipCity', 'ipIsp', 'ids', 'connectionType',
       'mcc', 'deviceModel', 'adSpaceReqUrl', 'cookie', 'privateMode', 'id',
       'isImpression', 'impressionType', 'impressionTime', 'day_of_week', 'h',
       'm', 's', 'eventCount', 'distEventTypes', 'eventTypeCounts', 'sin_time',
       'cos_time', 'sin_fet', 'cos_fet', 'sin_let', 'cos_let', 'sin_uid',
       'cos_uid', 'clientIp'],
      dtype='object')

In [10]:
df = pd.concat([df, pd.get_dummies(df['ipCity'])], axis=1);
del df['ipCity']

In [11]:
df = pd.concat([df, pd.get_dummies(df['ipRegion'])], axis=1);
del df['ipRegion']

In [12]:
df.columns

Index(['deliveryId', 'valid', 'adRelType', 'noAdReasonCode', 'forcedAd',
       'testAd', 'uid', 'dnt', 'marketId', 'operatorId',
       ...
       'Y8', 'Y9', 'Z1', 'Z2', 'Z3', 'Z4', 'Z5', 'Z6', 'Z7', 'Z8'],
      dtype='object', length=2492)